In [ ]:
# Import các thư viện cần thiết
from pylab import *  # Cung cấp công cụ tính toán và vẽ đồ thị
import networkx as nx  # Làm việc với đồ thị, sử dụng trong tính toán các chỉ số đồ thị
import math  # Thư viện toán học cơ bản
from numpy.linalg import inv  # Tính toán ma trận nghịch đảo
from sklearn.metrics import precision_recall_curve  # Tính Precision-Recall curve cho các dự đoán
from sklearn.preprocessing import MinMaxScaler  # Chuẩn hóa dữ liệu về khoảng giá trị [0, 1]
import copy  # Thư viện để sao chép đối tượng (deep copy)
from numpy import linalg as LA  # Thư viện tính toán đại số tuyến tính (Linear Algebra)
import csv  # Xử lý tệp CSV
import array  # Xử lý các cấu trúc mảng hiệu quả
import random  # Tạo số ngẫu nhiên, xáo trộn dữ liệu
import numpy  # Thư viện tính toán ma trận và xử lý số học
from deap import algorithms  # Các thuật toán di truyền (Genetic Algorithms)
from deap import base  # Cung cấp công cụ cơ bản để xây dựng thuật toán di truyền
from deap import creator  # Tạo lớp mới cho thuật toán di truyền
from deap import tools  # Bộ công cụ hỗ trợ cho thuật toán di truyền
from sklearn.metrics import roc_curve  # Tính ROC curve (True Positive Rate và False Positive Rate)
from sklearn.metrics import auc  # Tính diện tích dưới đường cong (AUC - Area Under Curve)
from sklearn.metrics import accuracy_score  # Tính độ chính xác (Accuracy) của mô hình
from sklearn.metrics import recall_score  # Tính độ nhạy (Recall)
from sklearn.metrics import f1_score  # Tính F1-score (trung bình điều hòa giữa Precision và Recall)
from sklearn.metrics import precision_score  # Tính độ chính xác (Precision)
from sklearn import linear_model  # Triển khai các mô hình hồi quy tuyến tính (Logistic Regression)

In [ ]:
def cross_validation(drug_drug_matrix, CV_num, seed):
    # Khởi tạo biến lưu số lượng liên kết (links) và các vị trí của liên kết, không phải liên kết
    link_number = 0
    link_position = []
    nonLinksPosition = []  # Danh sách tất cả các vị trí không phải liên kết (non-links)

    # Duyệt qua ma trận drug-drug để xác định các liên kết (link) và không phải liên kết
    for i in range(0, len(drug_drug_matrix)):
        for j in range(i + 1, len(drug_drug_matrix)):
            if drug_drug_matrix[i, j] == 1:  # Nếu là liên kết
                link_number = link_number + 1
                link_position.append([i, j])  # Lưu vị trí của liên kết
            else:  # Nếu không phải là liên kết
                nonLinksPosition.append([i, j])  # Lưu vị trí của không phải liên kết

    link_position = np.array(link_position)  # Chuyển danh sách các liên kết thành numpy array
    random.seed(seed)  # Đặt seed cho việc tạo số ngẫu nhiên
    index = np.arange(0, link_number)  # Tạo mảng các chỉ số từ 0 đến link_number - 1
    random.shuffle(index)  # Xáo trộn chỉ số ngẫu nhiên

    # Xác định số lượng fold cho cross-validation
    fold_num = link_number // CV_num
    print(fold_num)

    # Chạy vòng lặp cross-validation
    for CV in range(0, CV_num):
        print('*********round:' + str(CV) + "**********\n")

        # Chọn các chỉ số của các liên kết test trong fold hiện tại
        test_index = index[(CV * fold_num):((CV + 1) * fold_num)]
        test_index.sort()  # Sắp xếp lại các chỉ số của liên kết test
        testLinkPosition = link_position[test_index]  # Lấy vị trí của các liên kết test

        # Tạo bản sao của ma trận drug-drug và loại bỏ các liên kết trong ma trận train
        train_drug_drug_matrix = copy.deepcopy(drug_drug_matrix)
        for i in range(0, len(testLinkPosition)):
            train_drug_drug_matrix[testLinkPosition[i, 0], testLinkPosition[i, 1]] = 0  # Đặt giá trị của liên kết test thành 0
            train_drug_drug_matrix[testLinkPosition[i, 1], testLinkPosition[i, 0]] = 0  # Do ma trận đối xứng, xóa cả hai chiều

        # Danh sách vị trí test và non-link dùng cho việc dự đoán
        testPosition = list(testLinkPosition) + list(nonLinksPosition)

        # Xác định các tham số nội bộ (có thể là các tham số mô hình)
        weights, cf1, cf2 = internal_determine_parameter(copy.deepcopy(train_drug_drug_matrix))
        # cf1,cf2=internal_determine_parameter(copy.deepcopy(train_drug_drug_matrix))


        # Sử dụng phương pháp dự đoán kết hợp (ensemble) để dự đoán và tính toán kết quả
        [multiple_predict_matrix, multiple_predict_results] = ensemble_method(copy.deepcopy(drug_drug_matrix), train_drug_drug_matrix, testPosition)

        # Tính toán điểm số ensemble và các tham số (AUC, precision, recall...)
        ensemble_results, ensemble_results_cf1, ensemble_results_cf2 = ensemble_scoring(copy.deepcopy(drug_drug_matrix), multiple_predict_matrix, testPosition, weights, cf1, cf2)

        # Ghi kết quả dự đoán vào tệp
        for i in range(0, len(multiple_predict_results)):
            [auc_score, aupr_score, precision, recall, accuracy, f] = multiple_predict_results[i]
            file_results.write(auc_score + ' ' + aupr_score + ' ' + precision + ' ' + recall + ' ' + accuracy + ' ' + f + "\n")
            file_results.flush()

        # Ghi kết quả ensemble vào tệp
        [auc_score, aupr_score, precision, recall, accuracy, f] = ensemble_results
        file_results.write(auc_score + ' ' + aupr_score + ' ' + precision + ' ' + recall + ' ' + accuracy + ' ' + f + "\n")
        file_results.flush()

        [auc_score, aupr_score, precision, recall, accuracy, f] = ensemble_results_cf1
        file_results.write(auc_score + ' ' + aupr_score + ' ' + precision + ' ' + recall + ' ' + accuracy + ' ' + f + "\n")
        file_results.flush()

        [auc_score, aupr_score, precision, recall, accuracy, f] = ensemble_results_cf2
        file_results.write(auc_score + ' ' + aupr_score + ' ' + precision + ' ' + recall + ' ' + accuracy + ' ' + f + "\n")
        file_results.flush()

        # Ghi các trọng số vào tệp weights
        weights_str = ''
        for i in range(0, len(weights)):
            weights_str = weights_str + ' ' + str(weights[i])
        file_weights.write(weights_str + "\n")
        file_results.flush()
        file_weights.flush()

In [ ]:
class Topology:
    def topology_similarity_matrix(drug_drug_matrix):
        # Chuyển đổi ma trận drug_drug_matrix thành dạng ma trận numpy
        drug_drug_matrix = np.matrix(drug_drug_matrix)

        # Khởi tạo đồ thị từ ma trận drug_drug_matrix
        G = nx.from_numpy_array(drug_drug_matrix)

        # Số lượng thuốc trong ma trận
        drug_num = len(drug_drug_matrix)

        # Khởi tạo các ma trận tương tự nhau
        common_similarity_matrix = np.zeros(shape=(drug_num, drug_num))  # Ma trận tương tự hàng xóm chung
        AA_similarity_matrix = np.zeros(shape=(drug_num, drug_num))  # Ma trận tương tự Adamic-Adar
        RA_similarity_matrix = np.zeros(shape=(drug_num, drug_num))  # Ma trận tương tự Resource Allocation

        # Tính toán các giá trị trị riêng và vector trị riêng của ma trận drug_drug_matrix
        eigenValues, eigenVectors = linalg.eig(drug_drug_matrix)
        idx = eigenValues.argsort()[::-1]  # Sắp xếp các giá trị trị riêng theo thứ tự giảm dần
        eigenValues = eigenValues[idx[0]]  # Lấy giá trị trị riêng lớn nhất

        # Tính toán ma trận Katz similarity
        beta = 0.5 * (1 / eigenValues)  # Hệ số Katz
        Katz_similarity_matrix = inv(np.identity(drug_num) - beta * drug_drug_matrix) - np.identity(drug_num)

        # Tính toán ma trận Common Neighbor (hàng xóm chung)
        for i in range(0, drug_num):
            for j in range(i + 1, drug_num):
                commonn_neighbor = list(nx.common_neighbors(G, i, j))  # Lấy các hàng xóm chung của i và j
                common_similarity_matrix[i][j] = len(commonn_neighbor)  # Đếm số lượng hàng xóm chung
                AA_score = 0
                RA_score = 0
                for k in range(0, len(commonn_neighbor)):
                    # Tính điểm Adamic-Adar (AA)
                    AA_score += 1 / math.log(len(list(G.neighbors(commonn_neighbor[k]))))
                    # Tính điểm Resource Allocation (RA)
                    RA_score += 1 / len(list(G.neighbors(commonn_neighbor[k])))

                # Gán giá trị tương tự cho các ma trận AA và RA
                AA_similarity_matrix[i][j] = AA_score
                RA_similarity_matrix[i][j] = RA_score

                # Đảm bảo ma trận đối xứng
                common_similarity_matrix[j][i] = common_similarity_matrix[i][j]
                AA_similarity_matrix[j][i] = AA_similarity_matrix[i][j]
                RA_similarity_matrix[j][i] = RA_similarity_matrix[i][j]

        # Tính toán ma trận ACT similarity bằng cách sử dụng ma trận Laplacian
        D = np.diag(((drug_drug_matrix.sum(axis=1)).getA1()))  # Ma trận đường chéo với tổng dòng của drug_drug_matrix
        L = D - drug_drug_matrix  # Ma trận Laplacian
        LL = pinv(L)  # Ma trận nghịch đảo của L
        LL = np.matrix(LL)
        ACT_similarity_matrix = np.zeros(shape=(drug_num, drug_num))  # Khởi tạo ma trận ACT similarity

        # Tính toán ACT similarity cho các cặp thuốc
        for i in range(0, drug_num):
            for j in range(i + 1, drug_num):
                ACT_similarity_matrix[i][j] = 1 / (LL[i, i] + LL[j, j] - 2 * LL[i, j])  # Công thức ACT similarity
                ACT_similarity_matrix[j][i] = ACT_similarity_matrix[i][j]

        # Tính toán ma trận Random Walk with Restart (RWR) similarity
        D = np.diag(((drug_drug_matrix.sum(axis=1)).getA1()))  # Ma trận đường chéo với tổng dòng của drug_drug_matrix
        N = pinv(D) * drug_drug_matrix  # Ma trận chuẩn hóa của drug_drug_matrix
        alpha = 0.9  # Hệ số alpha cho Random Walk
        RWR_similarity_matrix = (1 - alpha) * pinv(np.identity(drug_num) - alpha * N)  # Tính ma trận RWR
        RWR_similarity_matrix = RWR_similarity_matrix + np.transpose(RWR_similarity_matrix)  # Đảm bảo ma trận đối xứng

        # Trả về các ma trận tương tự đã tính toán
        return np.matrix(common_similarity_matrix), np.matrix(AA_similarity_matrix), np.matrix(RA_similarity_matrix), np.matrix(Katz_similarity_matrix), np.matrix(ACT_similarity_matrix), np.matrix(RWR_similarity_matrix)

In [ ]:
def load_csv(filename, type):
    # Khởi tạo danh sách để lưu trữ dữ liệu ma trận từ CSV
    matrix_data = []

    # Mở tệp CSV để đọc
    with open(filename, 'r') as csvfile:
        # Đọc dữ liệu từ tệp CSV với phân cách là dấu phẩy
        csvreader = csv.reader(csvfile, delimiter=',')

        # Bỏ qua dòng đầu tiên (thường chứa tiêu đề cột)
        next(csvreader)

        # Duyệt qua từng dòng trong tệp CSV
        for row_vector in csvreader:
            # Kiểm tra kiểu dữ liệu, nếu 'int' thì chuyển đổi mỗi phần tử thành kiểu int
            if type == 'int':
                matrix_data.append(list(map(int, row_vector[1:])))  # Bỏ qua phần tử đầu tiên (thường là id hoặc tên)
            else:
                # Nếu không phải kiểu int, chuyển đổi thành kiểu float
                matrix_data.append(list(map(float, row_vector[1:])))  # Tương tự, bỏ qua phần tử đầu tiên

    # Trả về ma trận NumPy từ danh sách matrix_data
    return np.matrix(matrix_data)

In [ ]:
def modelEvaluation(real_matrix, predict_matrix, testPosition, featurename):
    # Khởi tạo danh sách để lưu trữ nhãn thực và xác suất dự đoán
    real_labels = []
    predicted_probability = []

    # Duyệt qua các vị trí kiểm tra để lấy nhãn thực và xác suất dự đoán
    for i in range(0, len(testPosition)):
        # Lấy nhãn thực từ ma trận real_matrix
        real_labels.append(real_matrix[testPosition[i][0], testPosition[i][1]])
        # Lấy xác suất dự đoán từ ma trận predict_matrix
        predicted_probability.append(predict_matrix[testPosition[i][0], testPosition[i][1]])

    # Chuyển danh sách thành mảng NumPy
    predicted_probability = np.array(predicted_probability)

    # Chuẩn hóa xác suất dự đoán về phạm vi [0, 1]
    min_val = predicted_probability.min()
    max_val = predicted_probability.max()
    predicted_probability = (predicted_probability - min_val) / (max_val - min_val)

    # Chuyển real_labels thành mảng NumPy
    real_labels = np.array(real_labels)
    predicted_probability = np.array(predicted_probability)

    # Tính precision, recall và các thresholds của PR curve
    precision, recall, pr_thresholds = precision_recall_curve(real_labels, predicted_probability)
    # Tính AUPR (Area Under the Precision-Recall curve)
    aupr_score = auc(recall, precision)

    # Tính F-measure cho từng threshold trong PR curve
    all_F_measure = np.zeros(len(pr_thresholds))
    for k in range(0, len(pr_thresholds)):
        if (precision[k] + precision[k]) > 0:  # Kiểm tra tránh chia cho 0
            all_F_measure[k] = 2 * precision[k] * recall[k] / (precision[k] + recall[k])
        else:
            all_F_measure[k] = 0

    # Tìm threshold cho F-measure tối đa
    max_index = all_F_measure.argmax()
    threshold = pr_thresholds[max_index]

    # Tính ROC curve và AUC cho ROC
    fpr, tpr, auc_thresholds = roc_curve(real_labels, predicted_probability)
    auc_score = auc(fpr, tpr)

    # Dự đoán nhãn dựa trên threshold tối ưu
    predicted_score = np.zeros(len(real_labels))
    predicted_score[predicted_probability > threshold] = 1

    # Tính các chỉ số đánh giá: F1, accuracy, precision, recall
    f = f1_score(real_labels, predicted_score)
    accuracy = accuracy_score(real_labels, predicted_score)
    precision = precision_score(real_labels, predicted_score)
    recall = recall_score(real_labels, predicted_score)

    # In ra kết quả đánh giá cho từng đặc trưng
    print('results for feature:' + featurename)
    print('************************AUC score:%.3f, AUPR score:%.3f, recall score:%.3f, precision score:%.3f, accuracy:%.3f, f-measure:%.3f************************' %
          (auc_score, aupr_score, recall, precision, accuracy, f))

    # Làm tròn các kết quả và trả về trong danh sách
    auc_score, aupr_score, precision, recall, accuracy, f = (
        "%.4f" % auc_score), ("%.4f" % aupr_score), ("%.4f" % precision), ("%.4f" % recall), ("%.4f" % accuracy), ("%.4f" % f)

    results = [auc_score, aupr_score, precision, recall, accuracy, f]

    return results

In [ ]:
def fitFunction(individual, parameter1, parameter2):
    # Tham số 'individual' chứa trọng số của các mô hình trong ensemble.
    # Đây là danh sách với mỗi phần tử là trọng số của từng mô hình trong bộ ensemble.

    real_labels = parameter1  # 'parameter1' là nhãn thực của dữ liệu.
    multiple_prediction = parameter2  # 'parameter2' là danh sách chứa các dự đoán từ các mô hình trong ensemble.

    # Khởi tạo mảng ensemble_prediction với tất cả giá trị là 0
    ensemble_prediction = np.zeros(len(real_labels))

    # Tính tổng các dự đoán từ các mô hình trong ensemble, nhân với trọng số của từng mô hình.
    for i in range(0, len(multiple_prediction)):
        ensemble_prediction = ensemble_prediction + individual[i] * multiple_prediction[i]

    # Tính Precision-Recall curve với các nhãn thực và dự đoán của ensemble.
    precision, recall, pr_thresholds = precision_recall_curve(real_labels, ensemble_prediction)

    # Tính AUPR (Area Under Precision-Recall curve)
    aupr_score = auc(recall, precision)

    # Trả về AUPR score dưới dạng tuple
    return (aupr_score),

In [ ]:
def getParamter(real_matrix, multiple_matrix, testPosition):
    # Kiểm tra nếu chưa tạo class FitnessMax, tạo mới nếu cần thiết.
    if not hasattr(creator, "FitnessMax"):
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))

    # Kiểm tra nếu chưa tạo class Individual, tạo mới nếu cần thiết.
    if not hasattr(creator, "Individual"):
        creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax)

    # Khởi tạo toolbox của DEAP để sử dụng cho thuật toán di truyền
    toolbox = base.Toolbox()

    # Hàm sinh ngẫu nhiên giá trị thuộc tính giữa 0 và 1
    toolbox.register("attr_float", random.uniform, 0, 1)

    # Khởi tạo số lượng cá thể trong population (số lượng trọng số trong ensemble)
    variable_num = len(multiple_matrix)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, variable_num)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    #################################################################################################
    # Tạo danh sách real_labels chứa nhãn thực từ 'real_matrix' tại các chỉ mục trong 'testPosition'
    real_labels = []
    for i in range(0, len(testPosition)):
        real_labels.append(real_matrix[testPosition[i][0], testPosition[i][1]])

    # Tạo danh sách 'multiple_prediction' chứa các dự đoán từ các mô hình trong 'multiple_matrix'
    multiple_prediction = []
    for i in range(0, len(multiple_matrix)):
        predicted_probability = []
        predict_matrix = multiple_matrix[i]
        for j in range(0, len(testPosition)):
            predicted_probability.append(predict_matrix[testPosition[j][0], testPosition[j][1]])

        # Chuẩn hóa giá trị dự đoán trong khoảng [0, 1]
        predicted_probability = np.array(predicted_probability)
        min_val = predicted_probability.min()
        max_val = predicted_probability.max()
        predicted_probability = (predicted_probability - min_val) / (max_val - min_val)

        # Thêm dự đoán vào danh sách multiple_prediction
        multiple_prediction.append(predicted_probability)

    #################################################################################################
    # Đăng ký hàm đánh giá 'evaluate' (sử dụng fitFunction) với 'real_labels' và 'multiple_prediction'
    toolbox.register("evaluate", fitFunction, parameter1=real_labels, parameter2=multiple_prediction)

    # Đăng ký các thao tác giao phối, đột biến, và chọn lọc trong thuật toán di truyền
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Tỉ lệ đột biến là 5%
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Khởi tạo một hạt giống ngẫu nhiên để đảm bảo tính tái lập của kết quả
    random.seed(0)

    # Khởi tạo population (100 cá thể)
    pop = toolbox.population(n=100)

    # Khởi tạo Hall of Fame (lưu trữ cá thể tốt nhất)
    hof = tools.HallOfFame(1)

    # Đăng ký các thống kê về population: trung bình, độ lệch chuẩn, min, max
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    # Chạy thuật toán di truyền (EA Simple) với các tham số cấu hình
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50,
                                   stats=stats, halloffame=hof, verbose=True)

    # Sắp xếp population theo giá trị fitness giảm dần và in ra cá thể tốt nhất
    pop.sort(key=lambda ind: ind.fitness, reverse=True)
    print(pop[0])

    # Trả về cá thể tốt nhất trong population
    return pop[0]

In [ ]:
class MethodHub():
    def neighbor_method(similarity_matrix, train_drug_drug_matrix):
        # Tính ma trận dự đoán dựa trên neighbor method.
        # Ma trận kết quả là tích giữa ma trận tương tác thuốc và ma trận tương tự.
        return_matrix = np.matrix(train_drug_drug_matrix) * np.matrix(similarity_matrix)

        # Tạo ma trận đường chéo D với các giá trị là tổng mỗi hàng của ma trận tương tự
        D = np.diag(((similarity_matrix.sum(axis=1)).getA1()))

        # Chuẩn hóa ma trận kết quả bằng cách nhân với nghịch đảo của D
        return_matrix = return_matrix * pinv(D)

        # Đảm bảo ma trận đối xứng bằng cách cộng với chuyển vị của nó
        return_matrix = return_matrix + np.transpose(return_matrix)

        # Trả về ma trận dự đoán
        return return_matrix

    def Label_Propagation(similarity_matrix, train_drug_drug_matrix):
        # Thực hiện phương pháp lan truyền nhãn (Label Propagation)
        alpha = 0.9  # Hệ số lan truyền, kiểm soát sự trộn lẫn thông tin mới và cũ

        # Chuyển ma trận đầu vào thành định dạng numpy matrix
        similarity_matrix = np.matrix(similarity_matrix)
        train_drug_drug_matrix = np.matrix(train_drug_drug_matrix)

        # Tạo ma trận đường chéo D từ tổng các hàng của ma trận tương tự
        D = np.diag(((similarity_matrix.sum(axis=1)).getA1()))

        # Tính ma trận chuẩn hóa N
        N = pinv(D) * similarity_matrix

        # Tính ma trận biến đổi dựa trên công thức Label Propagation
        transform_matrix = (1 - alpha) * pinv(np.identity(len(similarity_matrix)) - alpha * N)

        # Tính ma trận dự đoán
        return_matrix = transform_matrix * train_drug_drug_matrix

        # Đảm bảo ma trận đối xứng
        return_matrix = return_matrix + np.transpose(return_matrix)

        # Trả về ma trận dự đoán
        return return_matrix

    def generate_distrub_matrix(drug_drug_matrix):
        # Sinh ma trận thay đổi (disturbance matrix) bằng cách loại bỏ ngẫu nhiên một số liên kết
        A = np.matrix(drug_drug_matrix)
        [num, num] = A.shape  # Lấy kích thước ma trận

        # Lấy phần trên đường chéo chính của ma trận (để tránh tính trùng lặp)
        upper_A = np.triu(A, k=0)

        # Xác định các vị trí của các phần tử có giá trị 1 (các liên kết)
        [row_index, col_index] = np.where(upper_A == 1)

        # Tỷ lệ liên kết sẽ bị xóa
        ratio = 0.1
        select_num = int(len(row_index) * ratio)  # Số lượng liên kết bị xóa

        # Xáo trộn ngẫu nhiên các chỉ số
        index = arange(0, (upper_A.sum()).sum())
        # print(index.shape)

        random.seed(0)
        random.shuffle(index)

        # Chọn ngẫu nhiên các chỉ số cần loại bỏ
        select_index = index[0:select_num]

        # Tạo ma trận thay đổi delta_A
        delta_A = np.zeros(shape=(num, num))
        for i in range(0, select_num):
            delta_A[row_index[select_index[i]]][col_index[select_index[i]]] = 1
            delta_A[col_index[select_index[i]]][row_index[select_index[i]]] = 1

        # Trả về ma trận thay đổi, chỉ số hàng/cột và số lượng liên kết bị xóa
        return delta_A, row_index, col_index, select_num

    def disturb_matrix_method(train_drug_drug_matrix):
        # Xử lý ma trận bằng cách áp dụng sự thay đổi (disturbance)
        input_A = np.matrix(train_drug_drug_matrix)
        [num, num] = input_A.shape

        # Sinh ma trận thay đổi và lấy thông tin liên quan
        delta_A, row_index, col_index, select_num = MethodHub.generate_distrub_matrix(input_A)

        # Tạo ma trận thay đổi bằng cách trừ đi delta_A từ ma trận ban đầu
        A = input_A - delta_A

        # Tính giá trị riêng và vector riêng của ma trận
        eigenvalues, eigenvectors = LA.eig(A)
        num_eigenvalues = len(eigenvalues)

        # Tính sự thay đổi trong giá trị riêng do ma trận thay đổi
        delta_eigenvalues = np.zeros(num_eigenvalues)
        for i in range(0, num_eigenvalues):
            delta_eigenvalues[i] = ((np.transpose(eigenvectors[:, i]) @ delta_A @ eigenvectors[:, i]).item()) / (np.transpose(eigenvectors[:, i]) @ eigenvectors[:, i]).item()

        # Tái tạo lại ma trận tương tác từ giá trị riêng và vector riêng
        reconstructed_A = np.zeros(shape=(num, num))
        for i in range(0, num_eigenvalues):
            reconstructed_A = reconstructed_A + (eigenvalues[i] + delta_eigenvalues[i]) * eigenvectors[:, i] * np.transpose(eigenvectors[:, i])

        # Đảm bảo giữ nguyên các giá trị ban đầu trong ma trận
        reconstructed_A[np.where(input_A == 1)] = 1

        # Đảm bảo ma trận đối xứng
        return_matrix = reconstructed_A + np.transpose(reconstructed_A)

        # Trả về ma trận dự đoán sau khi áp dụng thay đổi
        return return_matrix

In [ ]:
def ensemble_method(drug_drug_matrix, train_drug_drug_matrix, testPosition):
    """
    Kết hợp nhiều phương pháp dựa trên các ma trận tương tự và lan truyền nhãn
    để dự đoán ma trận tương tác thuốc-drug (drug-drug interaction matrix).
    """

    # Tải các ma trận tương tự (similarity matrix) từ các nguồn khác nhau
    chem_sim_similarity_matrix = load_csv('dataset/chem_Jacarrd_sim.csv', 'float')  # Ma trận tương tự hóa học
    target_similarity_matrix = load_csv('dataset/target_Jacarrd_sim.csv', 'float')  # Ma trận tương tự mục tiêu
    transporter_similarity_matrix = load_csv('dataset/transporter_Jacarrd_sim.csv', 'float')  # Ma trận tương tự vận chuyển
    enzyme_similarity_matrix = load_csv('dataset/enzyme_Jacarrd_sim.csv', 'float')  # Ma trận tương tự enzyme
    pathway_similarity_matrix = load_csv('dataset/pathway_Jacarrd_sim.csv', 'float')  # Ma trận tương tự pathway
    indication_similarity_matrix = load_csv('dataset/indication_Jacarrd_sim.csv', 'float')  # Ma trận tương tự indication
    label_similarity_matrix = load_csv('dataset/sideeffect_Jacarrd_sim.csv', 'float')  # Ma trận tương tự side effect
    offlabel_similarity_matrix = load_csv('dataset/offsideeffect_Jacarrd_sim.csv', 'float')  # Ma trận tương tự off-label

    # Khởi tạo danh sách lưu kết quả và ma trận dự đoán từ các phương pháp khác nhau
    multiple_matrix = []  # Danh sách các ma trận dự đoán
    multiple_result = []  # Danh sách các kết quả đánh giá

    print('********************************************************')

    # Áp dụng phương pháp neighbor_method với từng ma trận tương tự
    predict_matrix = MethodHub.neighbor_method(chem_sim_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'chem_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    # Tương tự với các ma trận tương tự khác (target, transporter, enzyme, pathway, indication...)
    predict_matrix = MethodHub.neighbor_method(target_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'target_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    # Tiếp tục với các ma trận khác
    predict_matrix = MethodHub.neighbor_method(transporter_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'transporter_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.neighbor_method(enzyme_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'enzyme_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.neighbor_method(pathway_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'pathway_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.neighbor_method(indication_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'indication_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    # Áp dụng phương pháp Label Propagation với từng ma trận tương tự
    predict_matrix = MethodHub.Label_Propagation(chem_sim_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'chem_label')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.Label_Propagation(target_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'target_label')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    # Áp dụng với các ma trận khác (transporter, enzyme, pathway, indication...)
    predict_matrix = MethodHub.Label_Propagation(transporter_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'transporter_label')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.Label_Propagation(enzyme_similarity_matrix, train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'enzyme_label')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    # Tính các ma trận tương tự bằng Topology (Common, AA, RA, Katz, ACT, RWR)
    common_similarity_matrix, AA_similarity_matrix, RA_similarity_matrix, Katz_similarity_matrix, ACT_similarity_matrix, RWR_similarity_matrix = Topology.topology_similarity_matrix(train_drug_drug_matrix)

    # Áp dụng phương pháp neighbor_method với các ma trận từ Topology
    predict_matrix = MethodHub.neighbor_method(common_similarity_matrix, train_drug_drug_matrix)
    # predict_matrix=common_similarity_matrix
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'common_similarity_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.neighbor_method(AA_similarity_matrix, train_drug_drug_matrix)
    # predict_matrix=AA_similarity_matrix
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'AA_similarity_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    # Tương tự với RA, Katz, ACT, RWR
    predict_matrix = MethodHub.neighbor_method(RA_similarity_matrix, train_drug_drug_matrix)
    # predict_matrix=RA_similarity_matrix
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'RA_similarity_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.neighbor_method(Katz_similarity_matrix, train_drug_drug_matrix)
    # predict_matrix=Katz_similarity_matrix
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'Katz_similarity_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.neighbor_method(ACT_similarity_matrix, train_drug_drug_matrix)
    # predict_matrix=ACT_similarity_matrix
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'ACT_similarity_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    predict_matrix = MethodHub.neighbor_method(RWR_similarity_matrix, train_drug_drug_matrix)
    # predict_matrix=RWR_similarity_matrix
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'RWR_similarity_neighbor')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    # Áp dụng phương pháp disturb_matrix_method
    predict_matrix = MethodHub.disturb_matrix_method(train_drug_drug_matrix)
    results = modelEvaluation(drug_drug_matrix, predict_matrix, testPosition, 'disturb_matrix_label')
    multiple_result.append(results)
    multiple_matrix.append(predict_matrix)

    # Trả về danh sách các ma trận dự đoán và kết quả đánh giá
    return multiple_matrix, multiple_result

In [ ]:
def internal_determine_parameter(drug_drug_matrix):
    """
    Hàm này xác định các tham số cần thiết cho mô hình ensemble và logistic regression,
    dựa trên ma trận tương tác thuốc (drug-drug interaction matrix).
    """

    # Chia ma trận thành tập huấn luyện và tập kiểm tra (sử dụng phương pháp holdout)
    train_drug_drug_matrix, testPosition = holdout_by_link(copy.deepcopy(drug_drug_matrix), 0.2, 1)

    # Thực hiện phương pháp ensemble để tính toán các ma trận dự đoán và kết quả đánh giá
    [multiple_matrix, multiple_result] = ensemble_method(copy.deepcopy(drug_drug_matrix), train_drug_drug_matrix, testPosition)

    # Xác định trọng số cho từng mô hình trong ensemble bằng thuật toán di truyền
    weights = getParamter(copy.deepcopy(drug_drug_matrix), multiple_matrix, testPosition)

    # Khởi tạo các danh sách đầu vào (input_matrix) và đầu ra (output_matrix) cho logistic regression
    input_matrix = []  # Lưu trữ các đặc trưng từ các ma trận dự đoán
    output_matrix = []  # Lưu trữ nhãn thực tế từ ma trận gốc (drug_drug_matrix)

    # Duyệt qua các vị trí kiểm tra (testPosition)
    for i in range(0, len(testPosition)):
        vector = []
        # Lấy giá trị dự đoán từ từng ma trận trong `multiple_matrix` tại vị trí kiểm tra
        for j in range(0, len(multiple_matrix)):
            vector.append(multiple_matrix[j][testPosition[i][0], testPosition[i][1]])

        # Thêm đặc trưng và nhãn vào các danh sách tương ứng
        input_matrix.append(vector)
        output_matrix.append(drug_drug_matrix[testPosition[i][0], testPosition[i][1]])

    # Chuyển đổi danh sách đầu vào và đầu ra thành mảng NumPy để sử dụng trong Logistic Regression
    input_matrix = np.array(input_matrix)
    output_matrix = np.array(output_matrix)

    # Logistic Regression với penalty L1 (Lasso Regularization)
    clf1 = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6, solver='liblinear')
    clf1.fit(input_matrix, output_matrix)  # Huấn luyện mô hình Logistic Regression

    # Logistic Regression với penalty L2 (Ridge Regularization)
    clf2 = linear_model.LogisticRegression(C=1.0, penalty='l2', tol=1e-6)
    clf2.fit(input_matrix, output_matrix)  # Huấn luyện mô hình Logistic Regression

    # In thông báo hoàn thành xác định tham số
    print('*************************parameter determined*************************')

    # Trả về trọng số ensemble và hai mô hình logistic regression
    return weights, clf1, clf2

In [ ]:
def holdout_by_link(drug_drug_matrix, ratio, seed):
    """
    Chia ma trận tương tác thuốc thành tập huấn luyện và tập kiểm tra dựa trên liên kết.

    Tham số:
        - drug_drug_matrix: Ma trận tương tác thuốc-drug ban đầu (numpy array).
        - ratio: Tỷ lệ dữ liệu được sử dụng làm tập kiểm tra.
        - seed: Hạt giống ngẫu nhiên để đảm bảo tính tái lập.

    Trả về:
        - train_drug_drug_matrix: Ma trận huấn luyện với các liên kết trong tập kiểm tra được loại bỏ.
        - testPosition: Danh sách các vị trí trong tập kiểm tra và các vị trí không liên kết.
    """
    link_number = 0  # Số lượng liên kết trong ma trận
    link_position = []  # Danh sách các vị trí liên kết
    nonLinksPosition = []  # Danh sách các vị trí không có liên kết

    # Duyệt qua toàn bộ ma trận để tìm các vị trí liên kết và không liên kết
    for i in range(0, len(drug_drug_matrix)):
        for j in range(i + 1, len(drug_drug_matrix)):  # Xem xét phần trên đường chéo chính (ma trận đối xứng)
            if drug_drug_matrix[i, j] == 1:
                link_number += 1
                link_position.append([i, j])  # Lưu vị trí liên kết
            else:
                nonLinksPosition.append([i, j])  # Lưu vị trí không liên kết

    # Chuyển danh sách liên kết thành numpy array để xử lý dễ dàng
    link_position = np.array(link_position)

    # Khởi tạo hạt giống ngẫu nhiên để xáo trộn vị trí liên kết
    random.seed(seed)
    index = np.arange(0, link_number)  # Tạo mảng chỉ số từ 0 đến số liên kết - 1
    random.shuffle(index)  # Xáo trộn ngẫu nhiên các chỉ số

    # Chia tập liên kết thành tập huấn luyện và tập kiểm tra dựa trên tỷ lệ `ratio`
    train_index = index[(int(link_number * ratio) + 1):]  # Chỉ số của các liên kết trong tập huấn luyện
    test_index = index[0:int(link_number * ratio)]  # Chỉ số của các liên kết trong tập kiểm tra
    train_index.sort()  # Sắp xếp lại chỉ số
    test_index.sort()  # Sắp xếp lại chỉ số

    # Lấy danh sách các liên kết kiểm tra dựa trên `test_index`
    testLinkPosition = link_position[test_index]

    # Tạo bản sao của ma trận tương tác ban đầu để làm ma trận huấn luyện
    train_drug_drug_matrix = copy.deepcopy(drug_drug_matrix)

    # Xóa các liên kết trong tập kiểm tra khỏi ma trận huấn luyện
    for i in range(0, len(testLinkPosition)):
        train_drug_drug_matrix[testLinkPosition[i, 0], testLinkPosition[i, 1]] = 0
        train_drug_drug_matrix[testLinkPosition[i, 1], testLinkPosition[i, 0]] = 0  # Đảm bảo ma trận đối xứng

    # Tạo danh sách tập kiểm tra bao gồm cả các liên kết kiểm tra và các vị trí không liên kết
    testPosition = list(testLinkPosition) + list(nonLinksPosition)

    # Trả về ma trận huấn luyện và danh sách vị trí kiểm tra
    return train_drug_drug_matrix, testPosition

In [ ]:
def ensemble_scoring(real_matrix, multiple_matrix, testPosition, weights, cf1, cf2):
    """
    Tính toán điểm số của mô hình ensemble dựa trên các ma trận dự đoán, trọng số, và các mô hình logistic regression.

    Tham số:
        - real_matrix: Ma trận nhãn thực tế.
        - multiple_matrix: Danh sách các ma trận dự đoán từ các mô hình khác nhau.
        - testPosition: Vị trí kiểm tra (các cặp thuốc) để tính toán.
        - weights: Trọng số cho từng mô hình trong ensemble.
        - cf1, cf2: Các mô hình logistic regression đã huấn luyện.

    Trả về:
        - result: Kết quả dự đoán của ensemble (sử dụng trọng số).
        - result_cf1: Kết quả dự đoán từ logistic regression với regularization L1.
        - result_cf2: Kết quả dự đoán từ logistic regression với regularization L2.
    """

    # Khởi tạo danh sách lưu trữ nhãn thực tế
    real_labels = []
    for i in range(0, len(testPosition)):
        real_labels.append(real_matrix[testPosition[i][0], testPosition[i][1]])

    # Khởi tạo danh sách lưu trữ dự đoán từ các mô hình
    multiple_prediction = []
    for i in range(0, len(multiple_matrix)):
        predicted_probability = []  # Lưu trữ dự đoán từ ma trận hiện tại
        predict_matrix = multiple_matrix[i]

        # Lấy các giá trị dự đoán từ ma trận tại vị trí kiểm tra
        for j in range(0, len(testPosition)):
            predicted_probability.append(predict_matrix[testPosition[j][0], testPosition[j][1]])

        # Chuẩn hóa dự đoán về khoảng [0, 1]
        predicted_probability = np.array(predicted_probability)
        min_val = predicted_probability.min()
        max_val = predicted_probability.max()
        if max_val != min_val:
            predicted_probability = (predicted_probability - min_val) / (max_val - min_val)
        else:
            predicted_probability = np.zeros_like(predicted_probability)  # Nếu min_val == max_val, chuẩn hóa thành 0
        multiple_prediction.append(predicted_probability)

    # Tính toán dự đoán tổng hợp (ensemble) bằng cách kết hợp các dự đoán với trọng số
    ensemble_prediction = np.zeros(len(real_labels))
    for i in range(0, len(multiple_matrix)):
        ensemble_prediction += weights[i] * multiple_prediction[i]  # Trọng số cho mỗi dự đoán

    # Tính dự đoán từ các mô hình logistic regression (cf1 và cf2)
    ensemble_prediction_cf1 = np.zeros(len(real_labels))
    ensemble_prediction_cf2 = np.zeros(len(real_labels))
    for i in range(0, len(testPosition)):
        vector = []
        # Thu thập các dự đoán từ các mô hình tại vị trí kiểm tra
        for j in range(0, len(multiple_matrix)):
            vector.append(multiple_matrix[j][testPosition[i][0], testPosition[i][1]])

        vector = np.array(vector).reshape(1, -1)  # Chuyển thành mảng 2 chiều để làm đầu vào cho logistic regression
        ensemble_prediction_cf1[i] = (cf1.predict_proba(vector))[0][1]  # Dự đoán xác suất từ mô hình cf1
        ensemble_prediction_cf2[i] = (cf2.predict_proba(vector))[0][1]  # Dự đoán xác suất từ mô hình cf2

    # Chuẩn hóa dự đoán tổng hợp (ensemble_prediction) về khoảng [0, 1]
    ensemble_prediction = np.array(ensemble_prediction)
    min_val = ensemble_prediction.min()
    max_val = ensemble_prediction.max()
    if max_val != min_val:
        ensemble_prediction = (ensemble_prediction - min_val) / (max_val - min_val)
    else:
        ensemble_prediction = np.zeros_like(ensemble_prediction)

    # Tính toán các điểm số đánh giá dựa trên nhãn thực tế và dự đoán
    result = calculate_metric_score(real_labels, ensemble_prediction)  # Ensemble với trọng số
    result_cf1 = calculate_metric_score(real_labels, ensemble_prediction_cf1)  # Logistic regression L1
    result_cf2 = calculate_metric_score(real_labels, ensemble_prediction_cf2)  # Logistic regression L2

    # Trả về kết quả
    return result, result_cf1, result_cf2

In [ ]:
def calculate_metric_score(real_labels, predict_score):
    """
    Tính toán các chỉ số đánh giá (metrics) của mô hình dự đoán.

    Tham số:
        - real_labels: Danh sách nhãn thực tế (0 hoặc 1).
        - predict_score: Danh sách điểm số dự đoán (giá trị liên tục).

    Trả về:
        - results: Danh sách các chỉ số đánh giá gồm:
          + AUC (Area Under the ROC Curve)
          + AUPR (Area Under the Precision-Recall Curve)
          + Precision (Độ chính xác)
          + Recall (Độ nhạy)
          + Accuracy (Độ chính xác tổng thể)
          + F1 score (Harmonic mean của Precision và Recall)
    """

    # Tính Precision-Recall curve và AUPR (Area Under Precision-Recall Curve)
    precision, recall, pr_thresholds = precision_recall_curve(real_labels, predict_score)
    aupr_score = auc(recall, precision)  # Diện tích dưới Precision-Recall curve

    # Tính F-measure cho từng ngưỡng trong Precision-Recall curve
    all_F_measure = np.zeros(len(pr_thresholds))
    for k in range(0, len(pr_thresholds)):
        if (precision[k] + recall[k]) > 0:  # Tránh chia cho 0
            all_F_measure[k] = 2 * precision[k] * recall[k] / (precision[k] + recall[k])  # Công thức F1
        else:
            all_F_measure[k] = 0

    # Tìm ngưỡng (threshold) tối ưu dựa trên giá trị F-measure lớn nhất
    max_index = all_F_measure.argmax()
    threshold = pr_thresholds[max_index]

    # Tính ROC curve và AUC (Area Under ROC Curve)
    fpr, tpr, auc_thresholds = roc_curve(real_labels, predict_score)
    auc_score = auc(fpr, tpr)

    # Chuyển đổi điểm số dự đoán thành nhãn dự đoán (binary) dựa trên threshold tối ưu
    predicted_score = np.zeros(len(real_labels))
    predicted_score[predict_score > threshold] = 1  # Nếu điểm dự đoán lớn hơn ngưỡng, dự đoán là 1

    # Tính các chỉ số đánh giá khác
    f = f1_score(real_labels, predicted_score)  # F1 score
    accuracy = accuracy_score(real_labels, predicted_score)  # Độ chính xác tổng thể
    precision = precision_score(real_labels, predicted_score)  # Độ chính xác
    recall = recall_score(real_labels, predicted_score)  # Độ nhạy

    # In kết quả đánh giá
    print('results for feature:' + 'weighted_scoring')
    print(
        '************************AUC score:%.3f, AUPR score:%.3f, recall score:%.3f, precision score:%.3f, accuracy:%.3f************************' %
        (auc_score, aupr_score, recall, precision, accuracy)
    )

    # Làm tròn kết quả thành 4 chữ số thập phân
    auc_score, aupr_score, precision, recall, accuracy, f = (
        "%.4f" % auc_score, "%.4f" % aupr_score, "%.4f" % precision, "%.4f" % recall, "%.4f" % accuracy, "%.4f" % f
    )

    # Tổng hợp các chỉ số đánh giá
    results = [auc_score, aupr_score, precision, recall, accuracy, f]
    return results

In [ ]:
runtimes=20       # Thực hiện 20 lần chạy của kiểm định chéo 3-fold cho các bộ dự đoán cơ sở và mô hình ensemble

In [ ]:
drug_drug_matrix = load_csv('dataset/drug_drug_matrix.csv', 'int')
file_results_str="result/result_on_our_dataset_3CV"
weights_results_str="result/weights_on_our_dataset_3CV"
for seed in range(0, runtimes):
    file_results_path=file_results_str+"_"+str(seed)+".txt"
    weights_results_path=weights_results_str+"_"+str(seed)+".txt"
    file_results = open(file_results_path, "w")
    file_weights = open(weights_results_path, "w")
    cross_validation(drug_drug_matrix, 3, seed)